In [43]:
import pandas as pd
import numpy as np
import spacy
from nltk.corpus import movie_reviews
from nltk.tokenize import sent_tokenize
from nltk.wsd import lesk

# Esercizio 1.4 - Teoria di Hanks

La Teoria di Hanks propone di guardare al verbo ed alla sua valenza per trovare il significato di una frase.
Prevede un processo composto di tre passi:
1. estrazione dei _fillers_ del verbo
2. recupero dei _semantic types_ a partire dai _fillers_
3. clusterizzazione dei _semantic types_

Per questo esercizio è stato utilizzato il corpus _movie reviews_ di NLTK, contenente recensioni di film.

In [44]:
# import nltk
# nltk.download("movie_reviews")

In [45]:
# Scegliamo il verbo su cui operare
VERB = 'get'

## Estrazione dei fillers del verbo

Dal corpus recuperiamo 1500 frasi contenenti il verbo _get_.

In [46]:
def get_sentences(verb, corpus, num):
    """
    Given a corpus and a verb return the number of senteneces specified in the limit.

    :param verb: the verb as a string
    :param corpus: the corpus in which the search must be done
    :param num: the number of sentences to retrieve

    :return: the list of sentences containing the verb
    """
    i = 0
    sentences = np.array([])
    while len(sentences) < num:
        review = corpus.raw(corpus.fileids()[i])
        if verb in review.split():
            for s in sent_tokenize(review): # le recensioni contengono più frasi, le dividiamo considerandole singolarmente
                if verb in s.split():
                    sentences = np.append(sentences, s)
        i += 1
    return sentences


sentences = get_sentences(VERB, movie_reviews, 1500)
print('Numero di frasi recuperate: ', len(sentences))
sentences[:5]

Numero di frasi recuperate:  1500


array(['they get into an accident .',
       "now i personally don't mind trying to unravel a film every now and then , but when all it does is give me the same clue over and over again , i get kind of fed up after a while , which is this film's biggest problem .",
       'okay , we get it .',
       "you're more likely to get a kick out of her work in halloween h20 .",
       'all this to get to the familiar yet offensive " revelation " that sexual deviants are not , indeed , monsters but everyday people like you and me .'],
      dtype='<U887')

Effettuiamo __parsing__ e __disambiguazione__.

Partiamo dal __parsing__, il cui fine è quello di estrarre i _fillers_ del verbo dalle frasi.
Essendo il verbo scelto un verbo transitivo, consideriamo l'arità pari a 2 (soggetto ed oggetto), otteniamo così delle triple _(subj, verb, obj)_.

In [47]:
nlp = spacy.load("en_core_web_sm") # usiamo la libreria SpaCy per ottenere le dipendenze delle frasi
fillers = [] # inseriremo qui il soggetto e l'oggetto del verbo
index = [] # teniamo traccia dell'indice della frase per poterla recuperare nelle operazioni successive (WSD)
for i in range(len(sentences)):
    doc = nlp(str(sentences[i]))
    df = pd.DataFrame(data=[(token.dep_, token.head, [c for c in token.children]) for token in doc], index=[token.text for token in doc], columns=['token.dep_', 'token.head', 'token.children']) # indicizziamo tutto in un dataframe per recuperare i dati più velocemente
    childrens = df.loc[VERB, 'token.children'] # prendiamo i termini che sono in dipendenza del verbo indicato
    subj = None
    obj = None
    for c in childrens:
        try:
            c_dep = df.loc[str(c), 'token.dep_'] # prendiamo il valore della dipendenza
            try:
                c_dep = c_dep.iloc[0] # nel caso in cui la stessa parola compaia più volte nel dataframe prendiamo la prima occorrenza
            except AttributeError:
                pass
            # salviamo solo soggetto ed oggetto se presenti
            if str(c_dep) == 'nsubj':
                subj = str(c)
            if str(c_dep) == 'dobj':
                obj = str(c)
        except KeyError:
            # se ci sono più possibili triple nella stessa frase iteriamo su ognuna di esse
            for sub_child in c:
                c_dep = df.loc[str(sub_child), 'token.dep_'] # prendiamo il valore della dipendenza
                try:
                    c_dep = c_dep.iloc[0] # nel caso in cui la stessa parola compaia più volte nel dataframe ne prendiamo la prima occorrenza
                except AttributeError:
                    pass
                # salviamo solo soggetto ed oggetto se presenti
                if str(c_dep) == 'nsubj':
                    subj = str(sub_child)
                if str(c_dep) == 'dobj':
                    obj = str(sub_child)
            if subj and obj is not None:
                fillers.append((subj, VERB, obj))
                index.append(i)
                # azzerando i valori di subj ed obj possiamo catturare anche le altre triple della frase, altrimenti ci sarebbero due triple uguali
                subj = None
                obj = None
    if subj and obj is not None:
        fillers.append((subj, VERB, obj))
        index.append(i)

print('Numero di triple estratte:', len(fillers)) # le triple complete (senza nessun campo None) sono 266
fillers[:5]

Numero di triple estratte: 384


[('we', 'get', 'it'),
 ('that', 'get', 'laughs'),
 ('who', 'get', 'kicks'),
 ('we', 'get', 'money'),
 ('movie', 'get', 'star')]

In [48]:
# indicizziamo tutto in un dataframe per mostrare i risultati in maniera più organizzata
df = pd.DataFrame(data=fillers, index=index, columns=['subj','verb', 'obj'])
df.head()

,subj,verb,obj
2,we,get,it
5,that,get,laughs
6,who,get,kicks
8,we,get,money
9,movie,get,star


Procediamo ora con la __disambiguazione__ dei fillers, in questo modo avremo il synset di _WordNet_ che ne rappresenta il senso. L'algoritmo di WSD utilizato è quello di _Lesk_, che ritorna il synset più adatto al termine oppure _None_ nel caso in cui non riesca ad ottenerne uno.

In [49]:
syn_subj = []
syn_obj = []
for i, subj, obj in zip(df.index, df['subj'], df['obj']):
    sent = sentences[i].split()
    syn_subj.append(lesk(sent, subj))
    syn_obj.append(lesk(sent, obj))
df['subj syn'] = syn_subj
df['obj syn'] = syn_obj
df.head()

,subj,verb,obj,subj syn,obj syn
2,we,get,it,None,Synset('information_technology.n.01')
5,that,get,laughs,None,Synset('joke.n.01')
6,who,get,kicks,Synset('world_health_organization.n.01'),Synset('recoil.n.01')
8,we,get,money,None,Synset('money.n.03')
9,movie,get,star,Synset('movie.n.01'),Synset('star_topology.n.01')


Notiamo che in molte delle triple Lesk non riesce a trovare il senso del soggetto, mentre non ha grosse difficoltà a trovare quello dell'oggetto. Per avere un'idea contiamo il numero di elementi per cui è stato trovato il synset.

In [50]:
print('Numero di elementi non nulli fra i subj:  ', df['subj syn'].notnull().sum())
print('Numero di elementi non nulli fra gli obj: ', df['obj syn'].notnull().sum())
print('Lunghezza totale delle colonne:  ', len(df['subj syn']))

Numero di elementi non nulli fra i subj:   123
Numero di elementi non nulli fra gli obj:  335
Lunghezza totale delle colonne:   384


## Recupero dei semantic types

Passiamo adesso alla fase di estrazione dei __semantic types__. Li otteniamo accedendo a WordNet ed usando la funzione _lexname()_ presente in NLTK, che ritorna il super senso del termine.

In [51]:
df['super sense subj'] = df['subj syn'].apply(lambda x: x.lexname() if x is not None else None)
df['super sense obj'] = df['obj syn'].apply(lambda x: x.lexname() if x is not None else None)
df.head()

,subj,verb,obj,subj syn,obj syn,super sense subj,super sense obj
2,we,get,it,None,Synset('information_technology.n.01'),None,noun.cognition
5,that,get,laughs,None,Synset('joke.n.01'),None,noun.communication
6,who,get,kicks,Synset('world_health_organization.n.01'),Synset('recoil.n.01'),noun.group,noun.event
8,we,get,money,None,Synset('money.n.03'),None,noun.possession
9,movie,get,star,Synset('movie.n.01'),Synset('star_topology.n.01'),noun.communication,noun.cognition


## Clusterizzazione dei semantic types

Visualizziamo in maniera aggregata quali sono le categorie che compaiono con più frequenza fra i _semantic types_.

Prendiamo in esame la colonna dei _soggetti_.

In [52]:
print('Numero di categorie per il soggetto: ', len(df['super sense subj'].value_counts()))
df['super sense subj'].value_counts()

Numero di categorie per il soggetto:  24


noun.substance        35
noun.person           19
noun.group            10
adj.all               10
noun.communication     9
noun.cognition         8
noun.artifact          5
noun.quantity          4
noun.state             3
noun.location          2
noun.Tops              2
verb.motion            2
noun.event             2
verb.change            2
verb.creation          1
noun.feeling           1
verb.social            1
noun.time              1
verb.competition       1
noun.act               1
noun.food              1
noun.possession        1
verb.consumption       1
adv.all                1
Name: super sense subj, dtype: int64

Ora consideriamo quella degli _oggetti_.

In [53]:
print('Numero di categorie per l\'oggetto: ', len(df['super sense obj'].value_counts()))
df['super sense obj'].value_counts()

Numero di categorie per l'oggetto:  35


noun.communication    48
noun.cognition        34
noun.act              31
noun.person           26
noun.attribute        22
noun.state            21
noun.artifact         14
noun.possession       13
noun.quantity         11
adj.all               10
verb.possession        9
verb.communication     8
verb.cognition         8
noun.location          8
verb.perception        7
verb.social            6
noun.phenomenon        5
noun.event             5
verb.change            5
noun.feeling           5
verb.motion            5
noun.group             5
noun.time              4
noun.Tops              3
verb.competition       3
adv.all                3
verb.stative           3
verb.creation          3
noun.substance         2
verb.body              2
verb.consumption       2
noun.object            1
verb.contact           1
noun.animal            1
noun.food              1
Name: super sense obj, dtype: int64

Osservando le due distribuzioni si nota subito come la varianza di termini utilizzati sia molto maggiore per gli oggetti (35 categorie differenti).
I soggetti invece sono in minor numero e concentrati nelle categorie _noun.substance_ e _noun.person_.

Produciamo ora in output le frequenze delle triple _soggetto-verbo-oggetto_.

In [54]:
df.groupby(['super sense subj', 'verb', 'super sense obj']).size().sort_values(ascending=False)

super sense subj    verb  super sense obj
noun.substance      get   noun.state         4
                          noun.cognition     4
noun.group          get   noun.act           3
noun.communication  get   noun.cognition     3
adj.all             get   noun.attribute     3
                                            ..
noun.group          get   noun.event         1
                          noun.phenomenon    1
noun.location       get   noun.phenomenon    1
noun.person         get   adj.all            1
verb.social         get   noun.time          1
Length: 76, dtype: int64

Le frequenze delle triple mostrano valori piuttosto bassi, risaltano comunque alcuni significati tipici del verbo selezionato (_get_), fra tutti:
1. assumere uno stato (_noun.substance,get,noun.state_)
2. effettuare un'azione (_noun.group, get, noun.act_)
3. possesso (_noun.substance, get, noun.possession_)